In [1]:
import sys
# import dolfinx
import numpy as np
import matplotlib.pylab as plt
import slepc4py

from scipy.optimize import root
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc

from dolfinx import fem, io, plot, mesh
import ufl

import pyvista
pyvista.start_xvfb()


slepc4py.init(sys.argv)

In [2]:
# Define Geometry
L, H, B = 11e-3, 0.2e-3, 3e-3

geometry_mesh = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, H, B])],
    [20, 2, 5],
    cell_type=mesh.CellType.tetrahedron)

In [3]:
# Define vector space from geometry mesh
V = fem.VectorFunctionSpace(geometry_mesh, ("CG", 2))

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [4]:
fdim = geometry_mesh.topology.dim - 1

# Fixed z 
space, map = V.sub(2).collapse()
u_D1 = fem.Function(space)

# Assign all the displacements along y to be zero
with u_D1.vector.localForm() as loc:
    loc.set(0.)

# Locate facets where y = 0 or y = H
locate_dofs1 = fem.locate_dofs_geometrical((V.sub(1), space), lambda x: np.logical_or(np.isclose(x[1], 0), np.isclose(x[1], H)))

# Create Dirichlet BC
bc1 = fem.dirichletbc(u_D1, locate_dofs1, V.sub(1))

# Fixed y 
# Get sub space of y's
space_2, map_2 = V.sub(2).collapse()
u_D2 = fem.Function(space_2)

# Assign all the displacements along y to be zero
with u_D2.vector.localForm() as loc:
    loc.set(0.)

# Locate facets where y = 0 or y = H
locate_dofs2 = fem.locate_dofs_geometrical((V.sub(2), space_2), lambda x: np.logical_or(np.isclose(x[2], 0), np.isclose(x[2], B)))

# Create Dirichlet BC
bc2 = fem.dirichletbc(u_D2, locate_dofs2, V.sub(2))

In [5]:
# Define actual problem
E, nu = (5.4e10), (0.34)
rho = (7950.0)
mu = E/2./(1+nu)
lambda_ = E*nu/(1+nu)/(1-2*nu)


def epsilon(u):
    return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)


def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(3) \
           + 2*mu*epsilon(u)

T = fem.Constant(geometry_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0),
                    PETSc.ScalarType(0)))
k_form = ufl.inner(sigma(u), epsilon(v))*ufl.dx
m_form = rho*ufl.dot(u, v)*ufl.dx
k = fem.form(k_form)
m = fem.form(m_form)


# One of the big questions is: which boundary conditions do we apply?
# I am able to retrieve sensible results for either no boundary conditions, only
# boundary condition 1 and both boundary conditions. The resonance frequency
# changes quite a bit though.
K = fem.petsc.assemble_matrix(k, bcs=[bc1])
K.assemble()

M = fem.petsc.assemble_matrix(m, bcs=[bc1])
M.assemble()

In [6]:
# Define eigensolver and solve for eigenvalues
no_eigenvalues = 30 
target_frequency = 100000

eigensolver = SLEPc.EPS().create(geometry_mesh.comm)
eigensolver.setOperators(K, M)

eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
# tol = 1e-9
# eigensolver.setTolerances(tol=tol)

# Shift and invert mode
st = eigensolver.getST()
st.setType(SLEPc.ST.Type.SINVERT)
# target real eigenvalues
eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
# Set the target frequency
eigensolver.setTarget(target_frequency**2 * 2 * np.pi)
# Set no of eigenvalues to compute 
eigensolver.setDimensions(nev=no_eigenvalues)

# Solve the eigensystem
eigensolver.solve()


In [7]:
# Get the number of converged eigenpairs
evs = eigensolver.getConverged()

# Create dummy vectors for the eigenvectors to store the results in
vr, vi = K.createVecs()

k = 0
for i in range(evs):
    # e_val = eigensolver.getEigenpair(i, vr, vi)
    # Get the ith eigenvalue and eigenvector (vr and vi are placeholders for
    # the real and complex parts of the eigenvectors) they are then saved in
    # the input variables (vr, vi)
    eigenvalue = eigensolver.getEigenpair(i, vr, vi)
    # e_vec = eigensolver.getEigenvector(i, eh.vector)

    if (~np.isclose(eigenvalue.real, 1.0, atol=5)):
        # Calculation of eigenfrequency from real part of eigenvalue
        freq_3D = np.sqrt(eigenvalue.real)/2/np.pi

        eigenmode = fem.Function(V)
        eigenmode.vector[:] = vr.getArray()

        # plot_shape_with_resonance(x,y,mode_magnitude, L)
        # Create plotter and pyvista grid
        # p = pyvista.Plotter(off_screen=True)
        if k == 2:
            p = pyvista.Plotter()
            topology, cell_types, geometry = plot.create_vtk_mesh(V)
            grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


            # Attach vector values to grid and warp grid by vector
            grid["u"] = eigenmode.x.array.reshape((geometry.shape[0], 3))
            actor_0 = p.add_mesh(grid, style="wireframe", color="k")
            warped = grid.warp_by_vector("u", factor=5e-6)
            actor_1 = p.add_mesh(warped, show_edges=True)
            p.show_axes()
            if not pyvista.OFF_SCREEN:
               p.show()
            else:
                figure_as_array = p.screenshot("deflection.png")

        print(
            "Solid FE: {0:8.5f} [Hz]".format(freq_3D))
        k+=1


Solid FE: 57375.51707 [Hz]
Solid FE: 123252.59630 [Hz]


Widget(value="<iframe src='http://localhost:40537/index.html?ui=P_0x7f3b41ca2830_0&reconnect=auto' style='widt…

Solid FE: 124889.61146 [Hz]
Solid FE: 194002.22060 [Hz]
Solid FE: 241715.36558 [Hz]
Solid FE: 249632.53345 [Hz]
Solid FE: 300260.12175 [Hz]
Solid FE: 301374.94527 [Hz]
Solid FE: 334403.47718 [Hz]
Solid FE: 372340.82415 [Hz]
Solid FE: 374174.31890 [Hz]
Solid FE: 391189.19475 [Hz]
Solid FE: 400839.90380 [Hz]
Solid FE: 402687.36873 [Hz]
Solid FE: 439069.77600 [Hz]
Solid FE: 448521.79396 [Hz]
Solid FE: 452747.87079 [Hz]
Solid FE: 481956.95047 [Hz]
Solid FE: 504223.06112 [Hz]
Solid FE: 509611.99290 [Hz]
Solid FE: 519357.43611 [Hz]
Solid FE: 524665.92081 [Hz]
